# Training an EAGLE3 Draft Head for Cosmos-Reason2

This notebook walks through the full workflow for training an EAGLE3 speculative-decoding draft head on top of [nvidia/Cosmos-Reason2-8B](https://huggingface.co/nvidia/Cosmos-Reason2-8B).

**Workflow overview**

| Step | Description |
| :---: | :--- |
| 1 | Install dependencies |
| 2 | Authenticate with Hugging Face |
| 3 | Prepare training data from the Nemotron dataset |
| 4 | Inspect the bundled EAGLE3 config for Cosmos-Reason2 |
| 5 | Calibrate the draft vocabulary |
| 6 | Launch training |
| 7 | Export checkpoint for deployment |

> **Hardware requirement** – Cosmos-Reason2-8B requires at least one 80 GB GPU (e.g. H100/A100).
> Multi-GPU training is supported automatically via FSDP2 when more than one GPU is available.

## Step 1 – Install Dependencies

In [ ]:
# Install ModelOpt with Hugging Face support and the example-specific requirements.
# Run once; restart the kernel afterwards if running inside a fresh environment.
!pip install -U nvidia-modelopt[hf]
!pip install -r ../requirements.txt

## Step 2 – Authenticate with Hugging Face

Both `nvidia/Cosmos-Reason2-8B` and `nvidia/Nemotron-Post-Training-Dataset-v2` require you to
accept their respective licence agreements on the Hub before downloading.  
Log in once with your HF token (needs `read` scope):

In [ ]:
from huggingface_hub import login

# Paste your token or set the HF_TOKEN environment variable before running.
# login(token="hf_...")
login()  # interactive prompt

## Step 3 – Prepare Training Data

We use a curated subset of [nvidia/Nemotron-Post-Training-Dataset-v2](https://huggingface.co/datasets/nvidia/Nemotron-Post-Training-Dataset-v2)
(chat split) for training.  The `guides/nemotron_mapping.csv` file selects the specific rows to use.

The script streams only the required parquet shards and writes a conversation file in the
standard `jsonl` format expected by `launch_train.sh`.

In [ ]:
import os

# Run from the speculative_decoding root so that relative paths resolve correctly.
os.chdir("..")
print("Working directory:", os.getcwd())

In [ ]:
!python prepare_input_conversations/add_nemotron_chat.py \
    --mapping-file guides/nemotron_mapping.csv

In [ ]:
# Verify the output: the file should contain exactly 89 511 conversations.
data_path = "input_conversations/nemotron-chat.jsonl"

with open(data_path) as f:
    line_count = sum(1 for _ in f)

assert line_count == 89511, f"Expected 89511 lines, got {line_count}"
print(f"✓ {line_count} conversations written to {data_path}")

## Step 4 – EAGLE3 Config for Cosmos-Reason2

The `CR2_eagle_config.json` file is bundled alongside this notebook in the `guides/` directory.
It overrides the ModelOpt defaults with settings tuned for Cosmos-Reason2-8B (YaRN RoPE,
reduced draft vocabulary, FlexAttention).  No action is needed here — the cell below simply
prints the config for reference.

In [ ]:
import json
from pathlib import Path

config_path = Path("guides/CR2_eagle_config.json")
config = json.loads(config_path.read_text())

print(f"Config: {config_path}")
print(json.dumps(config, indent=4))

In [ ]:
# Path passed to launch_train.sh (relative to the speculative_decoding root).
EAGLE_CONFIG = "guides/CR2_eagle_config.json"
print("Eagle config path:", EAGLE_CONFIG)

## Step 5 – Calibrate the Draft Vocabulary *(optional)*

`CR2_eagle_config.json` sets `"draft_vocab_size": 32000`.  Using a compressed vocabulary
speeds up training and inference, but requires a one-time calibration step that produces a
token-mapping file (`d2t.pt`).  Skip this step if you prefer to train with the full vocabulary
(remove `--draft_vocab_cache` from the training command in Step 6 in that case).

In [ ]:
DRAFT_VOCAB_CACHE_DIR = "draft_vocab_cache"

!python scripts/calibrate_draft_vocab.py \
    --model nvidia/Cosmos-Reason2-8B \
    --data input_conversations/nemotron-chat.jsonl \
    --draft_vocab_size 32000 \
    --save_dir {DRAFT_VOCAB_CACHE_DIR}

## Step 6 – Train the EAGLE3 Draft Head

Training is launched via `launch_train.sh`, which internally calls `accelerate launch main.py`
and sets up FSDP2 automatically when multiple GPUs are available.

Key arguments used for Cosmos-Reason2:

| Argument | Value | Notes |
| :--- | :--- | :--- |
| `--model` | `nvidia/Cosmos-Reason2-8B` | Target VLM |
| `--data` | `input_conversations/nemotron-chat.jsonl` | Training conversations |
| `--eagle_config` | `guides/CR2_eagle_config.json` | Draft-head architecture |
| `--draft_vocab_cache` | `draft_vocab_cache/d2t.pt` | Token-mapping file from Step 5 *(optional)* |
| `--vlm_processor` | `nvidia/Cosmos-Reason2-8B` | VLM image processor |
| `--vlm_img_dir` | `data/` | Directory containing referenced images |
| `--training_seq_len` | `8192` | Max token length per sample |
| `--lr` | `1.5e-4` | Learning rate |
| `--num_epochs` | `20` | Training epochs |
| `--train_bs` | `1` | Per-device batch size |
| `--save_steps` | `1000` | Checkpoint frequency |
| `--ar_validate_steps` | `1000000` | Effectively disables in-training AR validation |

> **Tip** – Set `--ar_validate_steps` to a smaller value (e.g. `500`) to periodically measure
> acceptance rate on MT-Bench during training.

In [ ]:
OUTPUT_DIR = "ckpts/cosmos-reason2-8b-eagle3"
EAGLE_CONFIG = "guides/CR2_eagle_config.json"
DRAFT_VOCAB_CACHE = "draft_vocab_cache/Cosmos-Reason2-8B/d2t.pt"  # set to None to skip vocab compression

In [ ]:
draft_vocab_arg = f"--draft_vocab_cache {DRAFT_VOCAB_CACHE}" if DRAFT_VOCAB_CACHE else ""

# Outputs are streamed live.  Training 20 epochs on 89k samples with a single H100
# takes approximately 12–24 hours depending on sequence length distribution.
!export WANDB_MODE=disabled && OUTPUT_DIR={OUTPUT_DIR} \
  ./launch_train.sh \
  --model nvidia/Cosmos-Reason2-8B \
  --output_dir {OUTPUT_DIR} \
  --data input_conversations/nemotron-chat.jsonl \
  --lr 1.5e-4 \
  --num_epochs 20 \
  --train_bs 1 \
  --eagle_config {EAGLE_CONFIG} \
  {draft_vocab_arg} \
  --training_seq_len 8192 \
  --save_steps 1000 \
  --ar_validate_steps 1000000 \
  --vlm_processor nvidia/Cosmos-Reason2-8B \
  --vlm_img_dir data/

## Step 7 – Export Checkpoint for Deployment

After training completes, convert the ModelOpt checkpoint to the Hugging Face–compatible
format expected by vLLM.

In [ ]:
OUTPUT_DIR = "ckpts/cosmos-reason2-8b-eagle3"
EXPORT_PATH = "export/cosmos-reason2-8b-eagle3"

!python scripts/export_hf_checkpoint.py \
    --model_path {OUTPUT_DIR} \
    --export_path {EXPORT_PATH}

## Deployment

The exported checkpoint can be served directly with **vLLM**:

```bash
vllm serve nvidia/Cosmos-Reason2-8B \
    --host 0.0.0.0 \
    --port 8000 \
    --speculative-model export/cosmos-reason2-8b-eagle3 \
    --num-speculative-tokens 3 \
    --dtype bfloat16
```

Refer to the [vLLM speculative decoding docs](https://docs.vllm.ai/en/latest/features/spec_decode/) for the full list of options.